In [ ]:
from client import create_genai_client

client = create_genai_client()

In [ ]:
available_models = client.models.list()
model_names = [model.name for model in available_models]

In [ ]:
import ipywidgets as widgets
from IPython.display import display

model_selector = widgets.Dropdown(
    options=model_names,
    description="Model:",
)

display(model_selector)


def on_model_change(change):
    global MODEL_ID
    MODEL_ID = change.new
    print(f"Selected model: {MODEL_ID}")


model_selector.observe(on_model_change, names="value")

In [ ]:
import time
from pathlib import Path

from google.genai import types
from IPython.display import Markdown
from utils import read_text_from_file, write_gemini_text_to_markdown

time.sleep(5)

project_root = Path("../../../").resolve()

system_prompt_file = project_root / "prompts" / "system" / "meta-prompt.md"
try:
    system_prompt = read_text_from_file(system_prompt_file)
except FileNotFoundError:
    print("Error: The specified file was not found.")
except IOError as e:
    print(f"An error occurred while reading the file: {e}")

system_parts = [
    types.Part.from_text(text=system_prompt),
]

config = types.GenerateContentConfig(
    candidate_count=1,
    # frequency_penalty=0.2,
    max_output_tokens=4096,
    response_mime_type="text/plain",
    seed=12345,
    system_instruction=system_parts,
    temperature=0.3,
    top_p=1,
)

user_prompt_file = project_root / "prompts" / "user" / "prompt-generator.md"
try:
    user_prompt = read_text_from_file(user_prompt_file)
except FileNotFoundError:
    print("Error: The specified file was not found.")
except IOError as e:
    print(f"An error occurred while reading the file: {e}")

user_parts = types.Part.from_text(text=user_prompt)

contents = user_parts

response = client.models.generate_content(
    config=config,
    contents=contents,
    model=MODEL_ID,
)

response_file = project_root / "prompts" / "user" / "general-response.md"

write_gemini_text_to_markdown(response=response, output_path=response_file)

response_content = read_text_from_file(response_file)
Markdown(response_content)